# Readmission Risk Prediction with Random Forest

This notebook builds a Random Forest model to predict hospital readmission risk. It extends the Logistic Regression baseline by capturing non-linear relationships.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns

# Load data
df = pd.read_csv("../data/hospital_data.csv")
df.head()

In [ ]:
# Target
df["Readmission"] = df["Readmission"].str.strip().str.capitalize()
y = df["Readmission"].map({"No":0, "Yes":1})

# Features
X = df[["Age", "Gender", "Diagnosis", "Length_of_Stay", "Treatment", "Outcome"]]

In [ ]:
# Preprocessing
categorical_features = ["Gender", "Diagnosis", "Treatment", "Outcome"]
numeric_features = ["Age", "Length_of_Stay"]

preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features),
        ("num", "passthrough", numeric_features)
    ]
)

# Build pipeline with Random Forest
model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", RandomForestClassifier(n_estimators=200, random_state=42))
])

In [ ]:
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Fit model
model.fit(X_train, y_train)

In [ ]:
# Predictions
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:,1]

# Evaluation
print(classification_report(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_prob))

In [ ]:
# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt="d", cmap="Greens")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix - Random Forest")
plt.show()

In [ ]:
# Feature importance visualization
feature_names = model.named_steps['preprocessor'].transformers_[0][1].get_feature_names_out(categorical_features)
all_features = list(feature_names) + numeric_features

importances = model.named_steps['classifier'].feature_importances_
feat_imp = pd.Series(importances, index=all_features).sort_values(ascending=False)[:15]

plt.figure(figsize=(8,6))
sns.barplot(x=feat_imp.values, y=feat_imp.index)
plt.title("Top 15 Feature Importances - Random Forest")
plt.show()